# Merge all the dfs that were made
- So all the checks we did to see if a contig/protien was viral
- RdRp hmm matches
- Vibrant output
- Vibrant text search output
- BlastP output
- BlastN output (only one per contig)

In [3]:
# imports
import pandas as pd
import numpy as np

In [1]:
# open the RdRp search results
df_rdrp =  open_df('201228_RdRp_hits_eval_adapt.csv', ',')

# open vibrant results
df_vibrant = open_df('VIBRANT_annotations_all_contigs.txt','\t')

# open vibrant text search
df_vibrant_text = open_df('210119_viral_proteins_found_vibrant.csv', ',')

# open blastp
df_blastp = open_df('210120_blastp_output_w_desc_selectedviral.csv', ',')

# open blastn
df_blastn= open_df('210119_blastn_output_w_desc.csv', ',')

In [101]:
# give df_rdrp a column with contig names
df_rdrp['contig'] = df_rdrp.protein.str[:-2]

# give an extra column where we remove seed from the pfam number
df_rdrp['pfam_num'] = df_rdrp['hit'].str.replace('_seed','')

#Open df with the PFAM name -number combinations
df_pfam_names = open_df('rdrp_pfam_namenumber.csv', ',')

# replace the PFAM numbers in this df with the RdRp names
# write names and numbers to dict
pfam_dict = df_pfam_names.set_index('rdrp_name')['pfam_num'].to_dict()

# replace
df_rdrp['pfam_num'] = df_rdrp['pfam_num'].apply(str).replace(pfam_dict, regex=False)

# print head new df
df_rdrp.head()

,rdrp_name,pfam_num
0,PF00946,Mononeg_RNA_pol
1,PF07925,RdRP_5
2,PF00972,Flavi_NS5
3,PF04196,Bunya_RdRp
4,PF05919,Mitovir_RNA_pol


,protein,hit,evalue_hmm_rdrp,score_hmm_rdrp,bias_hmm_rdrp,start_hmm_rdrp,stop_hmm_rdrp,contig,pfam_num
0,Oak10_S14_1888_1,Mitovir_RNA_pol,4.300000e-23,81.6,0.0,74,373,Oak10_S14_1888,Mitovir_RNA_pol
1,Oak10_S14_4042_2,Mitovir_RNA_pol,3.900000e-10,38.9,0.2,588,815,Oak10_S14_4042,Mitovir_RNA_pol
2,Oak11_S15_14988_1,Mitovir_RNA_pol,1.000000e-15,57.7,0.0,3,416,Oak11_S15_14988,Mitovir_RNA_pol
3,Oak11_S15_2195_1,Mitovir_RNA_pol,4.800000e-75,253.5,0.0,68,2470,Oak11_S15_2195,Mitovir_RNA_pol
4,Oak12_S16_10149_1,Mitovir_RNA_pol,1.300000e-15,57.0,0.0,3,572,Oak12_S16_10149,Mitovir_RNA_pol


In [102]:
# rename vibrant column to contig
df_vibrant = df_vibrant.rename(columns={'scaffold': 'contig'})


# same for blastn
df_blastn = df_blastn.rename(columns={'qseqid': 'contig'})

# same for blastp
df_blastp = df_blastp.rename(columns={'qseqid': 'protein'})

In [112]:
# concatenate all the protein outpyut into one file
df_all_protein = df_rdrp.merge(df_vibrant,on='protein',how='outer').merge(df_vibrant_text,on='protein',how='outer').merge(df_blastp,on='protein',how='outer')

# make a new column contig so that we can merge with blastN results
df_all_protein['contig'] = df_all_protein.protein.str[:-2]

# merge protein results with blastn
df_all = df_all_protein.merge(df_blastn,on='contig',how='outer')

# write to file
df_all.to_csv('210120_all_resultsconcat.csv')

In [164]:
len(df_blastn_tocheck)

550

In [162]:
df_blastn_tocheck = df_all[~df_all.description_blastn.str.contains("virus", na=False) & ~df_all.description_blastp.str.contains("virus", na=False)]

# Write to file for manual check
df_blastn_tocheck.to_csv('210120_stringent_check.csv')

In [157]:
df_less_check = df_all[df_all.description_blastp.str.contains("virus", na=False) | df_all.description_blastn.str.contains("virus", na=False)]

In [163]:
# Write to file for manual check
df_less_check.to_csv('210120_less_check.csv')

In [1]:
def open_df(df, sep):
    # open the df
    df = pd.read_csv(df, sep=sep)
    # show top
    display(df.head())
    return df